In [1]:
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph

In [15]:
n = 100000
tau1 = 3
tau2 = 1.5
mu = 0.1
G = LFR_benchmark_graph(
    n, tau1, tau2, mu, average_degree=5, min_community=20, seed=10
)

In [16]:
communities = {frozenset(G.nodes[v]["community"]) for v in G}

In [17]:
print(len(communities))

951


In [8]:
import community as community_louvain

In [18]:
partion = community_louvain.best_partition(G)

In [19]:
#print(partion)
max(partion.values())

338